In [ ]:
from flask import Flask, jsonify, send_file, request, session
import pandas as pd
import mariadb
import re
from collections import Counter
from wordcloud import WordCloud
import matplotlib.pyplot as plt
from flask_cors import CORS
from konlpy.tag import Okt
import os

app = Flask(__name__)
CORS(app)

# MariaDB 연결 설정
def get_db_connection():
    conn = mariadb.connect(
        user="root",
        password="1234",
        host="localhost",
        port=3306,
        database="ansan"
    )
    return conn

# 제외 리스트
korean_stopwords = set([
    '있다', '하다', '되다', '수', '을', '에', '의', '가', '이', '으로', '도', '를', '는', '다', '그', '그리고', '에서',
    '지음', '있는', '통해', '위해', '대한', '가장', '모든', '한다', '라는', '여러', '같은', '많은', '대해'
])

# 형태소분석기
okt = Okt()

# 텍스트 전처리 함수
def preprocess_text(text):
    text = re.sub(r'\W', ' ', text)  # 특수 문자 제거
    words = okt.nouns(text)  # 명사 추출
    words = [word for word in words if word not in korean_stopwords and len(word) > 1]  # 1글자 제외
    return words

# 검색어 저장 함수
def save_search_keyword(user_id, gender, keyword):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("INSERT INTO search_keywords (user_id, gender, keyword) VALUES (?, ?, ?)", (user_id, gender, keyword))
    conn.commit()
    conn.close()

dummy_data = {
    'MALE': ['데이터베이스', '프로그래밍', '알고리즘', '네트워크', '리눅스', '보안', '서버', '웹', '정보처리기사', '소설', '음악', '국어', '스포츠'],
    'FEMALE': ['디자인', '마케팅', '글쓰기', '브랜딩', '포토샵', '일러스트', '사진', '편집', '메타인지', '자료', '심리학', '소설',
              '데이터','클라우드', '네트워크', '파이썬', '알고리즘', '정보', '정보처리', '정보처리기사', '포토샵', '포토샵', '포토샵',
              '그래픽', '그래픽', '3D', '3D', '2D', '음악', '네트워크', '수능', '메타버스', '메타버스', '웹', '보안']
}

# 책 상세 클라우드 생성
def create_wordcloud(id):
    try:
        conn = get_db_connection()
        query = f"SELECT book_detail FROM book WHERE id={id}"
        df = pd.read_sql(query, conn)
        conn.close()

        if df.empty:
            return None

        text = df.loc[0, 'book_detail']
        processed_text = preprocess_text(text)

        word_freq = Counter(processed_text)

        font_path = "D:\\python\\NanumGothic.ttf"

        wordcloud = WordCloud(width=800, height=400, background_color='white', font_path=font_path).generate_from_frequencies(word_freq)
        wordcloud_path = f"D:\\python\\wordcloud_{id}.png"
        wordcloud.to_file(wordcloud_path)
        return wordcloud_path
    except Exception:
        return None

# 성별 클라우드 생성
def create_wordcloud_for_gender(gender):
    conn = get_db_connection()
    cursor = conn.cursor()
    cursor.execute("SELECT keyword FROM search_keywords WHERE gender = ?", (gender,))
    keywords = [row[0] for row in cursor.fetchall()]
    conn.close()

    print(f"Keywords for {gender}: {keywords}")
    
    words = []
    for keyword in keywords:
        words.extend(preprocess_text(keyword))
    
    if gender in dummy_data:
        words.extend(dummy_data[gender])
    
    word_freq = Counter(words)
    print(f"Word frequencies for {gender}: {word_freq}")
    
    # 폰트 경로
    font_path = "D:\\python\\NanumGothic.ttf"

    wordcloud = WordCloud(width=800, height=400, background_color='white', font_path=font_path).generate_from_frequencies(word_freq)
    file_path = f"D:\\python\\wordcloud_{gender}.png"
    wordcloud.to_file(file_path)
    return file_path

# 책 상세 클라우드
@app.route('/api/wordcloud/<int:id>', methods=['GET'])
def get_wordcloud(id):
    wordcloud_path = create_wordcloud(id)
    if wordcloud_path and os.path.exists(wordcloud_path):
        return send_file(wordcloud_path, mimetype='image/png')
    else:
        return '클라우드 생성 실패!', 204

# 성별 클라우드
@app.route('/api/wordcloud/<gender>', methods=['GET'])
def get_wordcloud_for_gender(gender):
    try:
        file_path = create_wordcloud_for_gender(gender)
        return send_file(file_path, mimetype='image/png')
    except FileNotFoundError as e:
        print(f"File not found: {e}")
        return jsonify({"error": "File not found"}), 404

# 키워드 저장
@app.route('/api/save_keyword', methods=['POST'])
def save_keyword():
    data = request.json
    user_id = data.get("user_id")
    gender = data.get("gender")
    
    keyword = data.get('keyword')
    print(f"Saving keyword: {keyword} for user_id: {user_id} and gender: {gender}")
    if user_id and gender and keyword:
        save_search_keyword(user_id, gender, keyword)
        return jsonify({"status": "success"}), 200
    else:
        return jsonify({"error": "Invalid data"}), 400

if __name__ == '__main__':
    app.run(port=5001)
